In [4]:
/* sealed */ interface Entry { // Java 17부터는 sealed가 지원된다고 하더라 
    enum BinOp implements Entry { ADD, MULT }
    record Value(int number) implements Entry { } 
}

record Pair<A,B>(A a, B b) { }

interface ICalculator {
    boolean appendEntry(Entry e); // input either BinOp or Vaule
        // appends legal entry input and return true (returns false for illegal input)
        // updates last calculated value if possible
    boolean removeEntry(); // push the x button
        // removes the last input entry and return true (returns false if it is not possible)
        // updates last calculated value if necessary
    Entry.Value calculate();    // push the = button
        // calculate current entry and returns the result value (return null when it fails) 
        // add to history
        // clear current entries and nullify last calculated value
    Entry.Value lastValue(); // last calculated value
    List<Entry> entries();  // current entries
    List<Pair<Entry, Entry.Value>> history();  // history of calculations
    void clearEntries(); // clear the current entries, also clearing last calculate value
    void clearHistory(); // clear history
}

In [2]:
// ... Calculator implements ICalculator ...